In [ ]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords',quiet = 'True')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!pip install -q textblob
from textblob import TextBlob,Word
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/Sentiment Analysis on Student Feedback. (1).csv")
df.head()

,Timestamp,Course Code,Feedback,Sentiment,Previous Experience,Gender,Attendance,Course Difficulty,Study Hours (per week),Overall Satisfaction,Department
0,2024/03/10 10:13:18 PM GMT+5:30,ECE 1003,Course is kk.but teaching is bad,Negative,No,Female,Regular,Moderate,2,5,Yes
1,2024/03/10 10:24:15 PM GMT+5:30,CSE 1005,teaching is good,Positive,No,Female,Regular,Easy,11,9,Yes
2,2024/03/10 10:44:52 PM GMT+5:30,CSE 1005,It's good,Positive,No,Female,Regular,Challenging,7,8,Yes
3,2024/03/11 1:29:44 PM GMT+5:30,CSE 1005,The lecturers are knowledgeable and library fa...,Positive,Yes,Female,Irregular,Moderate,15,8,Yes
4,2024/03/11 4:06:44 PM GMT+5:30,MAT 1002,Teaching is almost good .,Neutral,Yes,Female,Occasional,Moderate,6,8,No


In [ ]:
df.shape

(83, 11)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Timestamp               83 non-null     object
 1   Course Code             83 non-null     object
 2   Feedback                83 non-null     object
 3   Sentiment               83 non-null     object
 4   Previous Experience     83 non-null     object
 5   Gender                  83 non-null     object
 6   Attendance              83 non-null     object
 7   Course Difficulty       83 non-null     object
 8   Study Hours (per week)  83 non-null     int64 
 9   Overall Satisfaction    83 non-null     int64 
 10  Department              83 non-null     object
dtypes: int64(2), object(9)
memory usage: 7.3+ KB


In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
# Extract date and time into separate columns
df['Date'] = df['Timestamp'].dt.date
df['Time'] = df['Timestamp'].dt.time
# Drop the timestamp column
new_df = df.drop(['Timestamp'],axis=1)

In [ ]:
new_df.head()

,Course Code,Feedback,Sentiment,Previous Experience,Gender,Attendance,Course Difficulty,Study Hours (per week),Overall Satisfaction,Department,Date,Time
0,ECE 1003,Course is kk.but teaching is bad,Negative,No,Female,Regular,Moderate,2,5,Yes,2024-03-10,22:13:18
1,CSE 1005,teaching is good,Positive,No,Female,Regular,Easy,11,9,Yes,2024-03-10,22:24:15
2,CSE 1005,It's good,Positive,No,Female,Regular,Challenging,7,8,Yes,2024-03-10,22:44:52
3,CSE 1005,The lecturers are knowledgeable and library fa...,Positive,Yes,Female,Irregular,Moderate,15,8,Yes,2024-03-11,13:29:44
4,MAT 1002,Teaching is almost good .,Neutral,Yes,Female,Occasional,Moderate,6,8,No,2024-03-11,16:06:44


In [ ]:
df['Sentiment'].value_counts()

Sentiment
Positive    47
Neutral     26
Negative    10
Name: count, dtype: int64

In [ ]:
new_df.isnull().sum()

Course Code               0
Feedback                  0
Sentiment                 0
Previous Experience       0
Gender                    0
Attendance                0
Course Difficulty         0
Study Hours (per week)    0
Overall Satisfaction      0
Department                0
Date                      0
Time                      0
dtype: int64

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install nltk
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Preprocessing the data further
def handle_contractions(text):
    # handling contractions
    contractions = {
        "n't": " not",
        "'s": " is",
        "'re": " are",
        "'ve": " have",
        "'d": " would",
        "'ll": " will",
        "'m": " am",
        "nw": "now"
    }
    words = text.split()
    for i in range(len(words)):
        if words[i] in contractions:
            words[i] = contractions[words[i]]
    return ' '.join(words)


def preprocess_text(text):

    # converting to lowercase for consistency in analysis
    text = text.lower()

    # removing nos or digits
    text = re.sub(r'\b[0-9]+\b\s*', '', text)

    # removing punctuations
    text = ''.join([char for char in text if char not in string.punctuation])

    # Correcting spellings TextBlob
    blob = TextBlob(text)
    corr_text = blob.correct()
    new_text = str(corr_text)

    # Applying tokenization
    tokens = word_tokenize(new_text)

    # Stopwords removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Handle contractions
    new_text = handle_contractions(' '.join(tokens))

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(new_text)]

    # Join tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text

# Apply preprocessing to the 'Feedback' column
new_df['Processed_Feedback'] = new_df['Feedback'].apply(preprocess_text)

In [ ]:
# Display the feedback column after preprocessing
new_df['Processed_Feedback']

0                                   course teaching bad
1                                         teaching good
2                                                  good
3           lecture knowledgeable library facility good
4                                  teaching almost good
                            ...                        
78       workload manageable assignment felt repetition
79    group project provided valuable opportunity co...
80    course could benefit update resource recent in...
81    instructor feedback assignment constructive he...
82    course fantastic lecturer averaged teaching mo...
Name: Processed_Feedback, Length: 83, dtype: object

In [ ]:
# subjectivity and polarity scores
def get_sentiment_scores(text):
    blob = TextBlob(text)
    return pd.Series([blob.sentiment.polarity, blob.sentiment.subjectivity])
sentiment_scores = new_df['Processed_Feedback'].apply(get_sentiment_scores)
new_df[['Sentiment_Score','Subjectivity_Score']] = sentiment_scores

new_df['Sentiment_label'] = new_df.apply(lambda row: 'Positive' if row['Sentiment_Score'] > 0 and
                                         row['Subjectivity_Score'] > 0.5 else
                                        'Negative' if row['Sentiment_Score'] < 0 and
                                         row['Subjectivity_Score'] > 0.5 else 'Neutral', axis=1)
new_df.drop('Sentiment',axis = 1,inplace = True)

In [ ]:
# Since we got the polarity and subjectivity scores, we will drop the original feedback column
new_df.drop('Feedback',axis = 1,inplace = True)

In [ ]:
new_df.head()

,Course Code,Previous Experience,Gender,Attendance,Course Difficulty,Study Hours (per week),Overall Satisfaction,Department,Date,Time,Processed_Feedback,Sentiment_Score,Subjectivity_Score,Sentiment_label
0,ECE 1003,No,Female,Regular,Moderate,2,5,Yes,2024-03-10,22:13:18,course teaching bad,-0.7,0.666667,Negative
1,CSE 1005,No,Female,Regular,Easy,11,9,Yes,2024-03-10,22:24:15,teaching good,0.7,0.600000,Positive
2,CSE 1005,No,Female,Regular,Challenging,7,8,Yes,2024-03-10,22:44:52,good,0.7,0.600000,Positive
3,CSE 1005,Yes,Female,Irregular,Moderate,15,8,Yes,2024-03-11,13:29:44,lecture knowledgeable library facility good,0.7,0.600000,Positive
4,MAT 1002,Yes,Female,Occasional,Moderate,6,8,No,2024-03-11,16:06:44,teaching almost good,0.7,0.600000,Positive


In [ ]:
# Shape of the current dataframe
new_df.shape

(83, 14)

In [ ]:
# Sentiment count as predicted by TextBlob library
new_df['Sentiment_label'].value_counts()

Sentiment_label
Positive    50
Neutral     27
Negative     6
Name: count, dtype: int64

In [ ]:
new_df.columns

Index(['Course Code', 'Previous Experience', 'Gender', 'Attendance',
       'Course Difficulty ', 'Study Hours (per week)', 'Overall Satisfaction',
       'Department', 'Date', 'Time', 'Processed_Feedback', 'Sentiment_Score',
       'Subjectivity_Score', 'Sentiment_label'],
      dtype='object')

In [ ]:
#CONVERTING CATEGORICAL VARIABLES INTO NUMERICAL VALUES
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
new_df['Previous Experience'] = label_encoder.fit_transform(new_df['Previous Experience'])
new_df['Gender'] = label_encoder.fit_transform(new_df['Gender'])
new_df['Attendance'] = label_encoder.fit_transform(new_df['Attendance'])
new_df['Course Difficulty '] = label_encoder.fit_transform(new_df['Course Difficulty '])
new_df['Sentiment_label'] = label_encoder.fit_transform(new_df['Sentiment_label'])
new_df['Department'] = label_encoder.fit_transform(new_df['Department'])

In [ ]:
new_df.head()

,Course Code,Previous Experience,Gender,Attendance,Course Difficulty,Study Hours (per week),Overall Satisfaction,Department,Date,Time,Processed_Feedback,Sentiment_Score,Subjectivity_Score,Sentiment_label
0,ECE 1003,0,0,2,3,2,5,1,2024-03-10,22:13:18,course teaching bad,-0.7,0.666667,0
1,CSE 1005,0,0,2,2,11,9,1,2024-03-10,22:24:15,teaching good,0.7,0.600000,2
2,CSE 1005,0,0,2,0,7,8,1,2024-03-10,22:44:52,good,0.7,0.600000,2
3,CSE 1005,1,0,0,3,15,8,1,2024-03-11,13:29:44,lecture knowledgeable library facility good,0.7,0.600000,2
4,MAT 1002,1,0,1,3,6,8,0,2024-03-11,16:06:44,teaching almost good,0.7,0.600000,2


In [ ]:
#dropping unecessary columns to fit a model
new_df.drop(['Date','Time','Processed_Feedback','Course Code'],axis = 1,inplace = True)

In [ ]:
new_df.head()

,Previous Experience,Gender,Attendance,Course Difficulty,Study Hours (per week),Overall Satisfaction,Department,Sentiment_Score,Subjectivity_Score,Sentiment_label
0,0,0,2,3,2,5,1,-0.7,0.666667,0
1,0,0,2,2,11,9,1,0.7,0.600000,2
2,0,0,2,0,7,8,1,0.7,0.600000,2
3,1,0,0,3,15,8,1,0.7,0.600000,2
4,1,0,1,3,6,8,0,0.7,0.600000,2


In [ ]:
#MODEL BUILDING - FITTING A MULTIPLE LINEAR REGRESSION MODEL
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
x = new_df.drop(['Previous Experience','Gender','Attendance','Course Difficulty ','Study Hours (per week)','Overall Satisfaction','Department','Sentiment_Score','Sentiment_label'], axis=1)  # Independent variables
y = new_df['Subjectivity_Score'] # Target variable

#Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Fitting the Multiple Regression model
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
#CALCULATING THE REQUIRED METRICS FOR THE MODEL
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Mean Squared Error: 3.4440159005512924e-33
Root Mean Squared Error: 5.868573847666307e-17
Mean Absolute Error: 3.591898020846095e-17
R-squared: 1.0


In [ ]:
n = x_test.shape[0]  # Number of observations
p = x_test.shape[1]  # Number of predictors
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [ ]:
adjusted_r2

1.0

In [ ]:
#FITTING THE MODEL WITH RESPECT TO THREE VARIABLES
x=new_df.drop(['Gender','Course Difficulty ','Study Hours (per week)'],axis=1)
y=new_df['Subjectivity_Score']

In [ ]:
#Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Fitting the Multiple Regression model
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
#CALCULATING THE REQUIRED METRICS FOR THE MODEL
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Mean Squared Error: 4.225212645711771e-30
Root Mean Squared Error: 2.055532204980445e-15
Mean Absolute Error: 1.681222082942386e-15
R-squared: 1.0


In [ ]:
n = x_test.shape[0]  # Number of observations
p = x_test.shape[1]  # Number of predictors
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [ ]:
adjusted_r2

1.0